<a href="https://colab.research.google.com/github/kywch/deeplabv3-semantic-scene-segmentation/blob/master/Google-colab_scene_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# check the pytorch installation
import torch
print(torch.cuda.is_available())       # Should be True
print(torch.backends.cudnn.enabled)    # Should be True

True
True


In [0]:
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import os
import csv
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import cv2

### Cloning the github repository to get the necessary files


In [4]:
!git clone https://github.com/kywch/deeplabv3-semantic-scene-segmentation.git

Cloning into 'deeplabv3-semantic-scene-segmentation'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 107 (delta 30), reused 32 (delta 10), pack-reused 36
Receiving objects: 100% (107/107), 258.72 MiB | 37.03 MiB/s, done.
Resolving deltas: 100% (40/40), done.
Checking out files: 100% (25/25), done.


In [5]:
# git repo files are placed under `deeplabv3-semantic-scene-segmentation
!ls

deeplabv3-semantic-scene-segmentation  sample_data


In [0]:
# loading the files from git repo
import sys

sys.path.append("deeplabv3-semantic-scene-segmentation/deeplabv3")
sys.path.append("deeplabv3-semantic-scene-segmentation/deeplabv3/model")
from deeplabv3 import DeepLabV3

sys.path.append("deeplabv3-semantic-scene-segmentation/deeplabv3/utils")
from utils import label_img_to_color


### Download the images to process

In [9]:
!wget https://www.dropbox.com/s/s5pmkr2gqh8dx5p/images307.zip

--2020-05-29 02:17:44--  https://www.dropbox.com/s/s5pmkr2gqh8dx5p/images307.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/s5pmkr2gqh8dx5p/images307.zip [following]
--2020-05-29 02:17:45--  https://www.dropbox.com/s/raw/s5pmkr2gqh8dx5p/images307.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc93ef772b23135463505e21f901.dl.dropboxusercontent.com/cd/0/inline/A4kBcD1Pj6tPUDi7ynK_u5XfuRP6BIoGe5AkLdbKIPQQbyIVYS5jSUxoW747scNidE1UfkHCfoga42ustxOtaOvnjGk_cJZ9BZzKKBB2yfzM6uSeH_FCQlsmTinppieLrgk/file# [following]
--2020-05-29 02:17:45--  https://uc93ef772b23135463505e21f901.dl.dropboxusercontent.com/cd/0/inline/A4kBcD1Pj6tPUDi7ynK_u5XfuRP6BIoGe5AkLdbKIPQQbyIVYS5jSUxoW747scNidE1UfkHCfoga42ustxOtaOvnjGk_cJZ9BZzKKBB2yfzM

In [17]:
#!ls
!unzip -q images307.zip

replace images307/Amazing_Grace_086.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
lib_dir = 'deeplabv3-semantic-scene-segmentation/deeplabv3/'

# Specify your output directory
input_dir = "images307"
output_dir = "output"

In [19]:
network = DeepLabV3("eval_val", project_dir="deeplabv3-semantic-scene-segmentation/deeplabv3", 
                    pretrained_file=lib_dir+"resnet18-5c106cde.pth").cuda()

pretrained resnet, 18


In [20]:
network.load_state_dict(torch.load(lib_dir+"model_13_2_2_2_epoch_580.pth"))

<All keys matched successfully>

In [0]:
class DatasetVal(torch.utils.data.Dataset):
    def __init__(self, input_path, output_path):
        self.img_dir = input_path
        self.img_h = 1024
        self.img_w = 2048
        self.output_dir = output_path
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        self.debug_dir = output_path + '/debug/'
        if not os.path.exists(self.debug_dir):
            os.makedirs(self.debug_dir)
        
        self.examples = []
        file_names = os.listdir(self.img_dir)
        for file_name in file_names:
            if file_name.lower().endswith(('.jpg', '.png', '.jpeg', '.BMP')):
                img_id = file_name.split('.')[0]
                img_path = self.img_dir + '/' + file_name
                example = {}
                example["img_path"] = img_path
                example["img_id"] = img_id
                self.examples.append(example)
        #print(self.examples)

        self.num_examples = len(self.examples)
        print(str(self.num_examples) + " images were identified from the input directory.")
        
    def __getitem__(self, index):
        example = self.examples[index]

        img_id = example["img_id"]
        img_path = example["img_path"]
        img = cv2.imread(img_path, -1) # (shape: (1024, 2048, 3))

        # normalize the img (with the mean and std for the pretrained ResNet):
        img = img/255.0
        img = img - np.array([0.485, 0.456, 0.406])
        img = img/np.array([0.229, 0.224, 0.225]) # (shape: (512, 1024, 3))
        img = np.transpose(img, (2, 0, 1)) # (shape: (3, 512, 1024))
        img = img.astype(np.float32)

        # convert numpy -> torch:
        img = torch.from_numpy(img) # (shape: (3, 512, 1024))
        #label_img = torch.from_numpy(label_img) # (shape: (512, 1024))
        print(img_id)
        return (img, img_id)

    def __len__(self):
        return self.num_examples


In [29]:
val_dataset = DatasetVal(input_path=input_dir, output_path=output_dir)

208 images were identified from the input directory.


In [30]:
val_dataset.examples

[{'img_id': 'washingtonmarvin1',
  'img_path': 'images307/washingtonmarvin1.jpg'},
 {'img_id': 'AnneArundel3', 'img_path': 'images307/AnneArundel3.jpg'},
 {'img_id': 'grow2', 'img_path': 'images307/grow2.jpg'},
 {'img_id': 'Earth1', 'img_path': 'images307/Earth1.jpg'},
 {'img_id': 'PattersonPark029', 'img_path': 'images307/PattersonPark029.jpg'},
 {'img_id': 'CBF3', 'img_path': 'images307/CBF3.jpg'},
 {'img_id': 'BaltParksCarmine2',
  'img_path': 'images307/BaltParksCarmine2.jpg'},
 {'img_id': 'Franklin1', 'img_path': 'images307/Franklin1.jpg'},
 {'img_id': 'kensington', 'img_path': 'images307/kensington.jpg'},
 {'img_id': 'washwatts1', 'img_path': 'images307/washwatts1.jpg'},
 {'img_id': 'AVAM_02', 'img_path': 'images307/AVAM_02.jpg'},
 {'img_id': 'washmeridian3', 'img_path': 'images307/washmeridian3.jpg'},
 {'img_id': 'Mt_Washington_Arboretum028',
  'img_path': 'images307/Mt_Washington_Arboretum028.jpg'},
 {'img_id': 'WhitWalker_TKF049',
  'img_path': 'images307/WhitWalker_TKF049.jpg

In [0]:
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=1, shuffle=False,
                                         num_workers=1)

In [32]:
network.eval()

DeepLabV3(
  (resnet): ResNet_BasicBlock_OS8(
    (resnet): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d

In [33]:
count = 0

#produce labeled image
for step, (imgs, img_ids) in enumerate(val_loader):
    #print(img_ids)
    with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)
        imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
        outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))
  
        ########################################################################
        # save data for visualization:
        ########################################################################
        
        outputs = outputs.data.cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
        pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
        pred_label_imgs = pred_label_imgs.astype(np.uint8)

        for i in range(pred_label_imgs.shape[0]):
            if i == 0:
                pred_label_img = pred_label_imgs[i] # (shape: (img_h, img_w))
                img_id = img_ids[i]
                img = imgs[i] # (shape: (3, img_h, img_w))

                img = img.data.cpu().numpy()
                img = np.transpose(img, (1, 2, 0)) # (shape: (img_h, img_w, 3))
                img = img*np.array([0.229, 0.224, 0.225])
                img = img + np.array([0.485, 0.456, 0.406])
                img = img*255.0
                img = img.astype(np.uint8)

                pred_label_img_color = label_img_to_color(pred_label_img)
                overlayed_img = 0.35*img + 0.65*pred_label_img_color
                overlayed_img = overlayed_img.astype(np.uint8)
                #producing original predicted images
                cv2.imwrite(val_dataset.output_dir + "/" + img_id + "_pred_label.png", pred_label_img )
                
                # produce debugging info for the first 10 images
                if count < 100:
                    #producing colored predicted images
                    cv2.imwrite(val_dataset.debug_dir + "/" + img_id + "_pred_label_colorimg.png", pred_label_img_color )
                    #producing overlayed images
                    cv2.imwrite(val_dataset.debug_dir + "/" + img_id + "_overlayed.png", overlayed_img)
                

        ########################################################################
        # quick counting
        ########################################################################
        count += 1
                

washingtonmarvin1
AnneArundel3
grow2


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2796: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Earth1
PattersonPark029
CBF3
BaltParksCarmine2
Franklin1
kensington
washwatts1
AVAM_02
washmeridian3
Mt_Washington_Arboretum028
WhitWalker_TKF049
Marian_House16
WhitWalker_TKF007
UMCP2
BaltParkEastside
newborn3
Frederick2
jeremys2
ASTT_13
GreenspacesPenn
stanthonys1
StAnthony_s123
CBF1
Mt_Washington_Arboretum029
washwatts2
Epiphany3
APA2
Childrens2
Friends2
metro3
Frederick3
Village_Learning_Place25
hopkins1
Greenspaces1
newborn1
Providence_Hospital_20
upenn3
UMBC-Beuys062
leon2
UMCP3
java1
ARC2
StLuke_s_Bethesda_13
stpauls3
upenn1
margo3
APA3
BaltParkGreenmount
league2
UMBC-Beuys054
Thanksgiving_Place_65
meridian1
hopkins2
kids3
grow3
ARC_of_PG
Amazing_Grace_087
rocco_buda_after
jeremys3
hopkins3
kernan1
Providence_Hospital_31
A_Wider_Circle_017
Friends3
Bates3
NDC1
Providence_Hospital_17
Govans3
MD2
MD3
Greenspaces3
AnneArundel1
Epiphany1
Annapolis_Waterworks1
ARC_of_PG_Dedication_028
MD1
Bates2
Annapolis_Waterworks3
StLuke_s_labyrinth_from_the_churchyard
Amazing_Grace_086
Thanksgivi

In [0]:
# generate the summary csv file from the labeled files
import glob
output_files = glob.glob(val_dataset.output_dir + "/*pred_label.png")

In [35]:
print(val_dataset.output_dir)
print(output_files)

output
['output/AVAM_02_pred_label.png', 'output/BaltParksKeySchool_pred_label.png', 'output/Childrens1_pred_label.png', 'output/APA2_pred_label.png', 'output/PattersonPark029_pred_label.png', 'output/league1_pred_label.png', 'output/ASTT_17_pred_label.png', 'output/Epiphany1_pred_label.png', 'output/jeremys1_pred_label.png', 'output/BaltParkReservoir_pred_label.png', 'output/MD2_pred_label.png', 'output/BaltParksDSC_pred_label.png', 'output/BaltClayworks3_pred_label.png', 'output/washingtonmarvin1_pred_label.png', 'output/Georgetowneast_pred_label.png', 'output/washwatts1_pred_label.png', 'output/ASTT_07_pred_label.png', 'output/Congressional_pred_label.png', 'output/margo2_pred_label.png', 'output/jessup2_pred_label.png', 'output/sharpe2_pred_label.png', 'output/rocco_buda_after_pred_label.png', 'output/Amazing_Grace_087_pred_label.png', 'output/Thanksgiving_Place_69_pred_label.png', 'output/WCI_007_pred_label.png', 'output/unity2_pred_label.png', 'output/WhitWalker_TKF049_pred_label

In [0]:
# deeplabv3 - cityscape labels

class_names = ['pcnt_road', 'pcnt_sidewalk', 'pcnt_building', 'pcnt_wall', 'pcnt_fence',
    'pcnt_pole', 'pcnt_traffic_light', 'pcnt_traffic_sign', 'pcnt_vegetation', 'pcnt_terrain',
    'pcnt_sky', 'pcnt_person', 'pcnt_rider', 'pcnt_car', 'pcnt_truck',
    'pcnt_bus', 'pcnt_train', 'pcnt_motorcycle', 'pcnt_bicycle', 'pcnt_unlabeled']
num_class = len(class_names)


In [37]:
# quick check

file = output_files[0]

print(os.path.basename(file).split('_pred_label.png')[0])

pred_label_img = cv2.imread(file, -1)
# count how many pixels in pred_label_img which are of object class trainId:
pixel_ratio = np.zeros(num_class)
for cate_id in range(num_class):
    pixel_ratio[cate_id] = np.sum(np.equal(pred_label_img, cate_id))/pred_label_img.size

print(pixel_ratio.tolist())

AVAM_02
[0.0, 0.00018328445747800586, 0.09368408190564388, 0.0, 0.0, 0.011353989813242784, 0.0, 0.0014727066934197663, 0.5836741781139064, 0.19049364613880743, 0.04078239954725524, 0.06312059474198693, 0.0, 0.004688223491279519, 3.8586201574317025e-05, 0.0, 0.0, 0.0, 0.0, 0.010508308895405669]


In [0]:
#save category ratios to csv file
with open('00_object_ratios.csv', 'w') as output:
    writer = csv.writer(output)
    writer.writerow(['image_name'] + class_names)
    
    for file in output_files:
        img_name = os.path.basename(file).split('_pred_label.png')[0]
        pred_label_img = cv2.imread(file, -1)
        
        # count how many pixels in pred_label_img which are of object class trainId:
        pixel_ratio = np.zeros(num_class)
        for cate_id in range(num_class):
            pixel_ratio[cate_id] = np.sum(np.equal(pred_label_img, cate_id))/pred_label_img.size
        
        # write to the csv file
        writer.writerow([img_name] + pixel_ratio.tolist())


In [39]:
!ls

 00_object_ratios.csv			 images307.zip	       output
 deeplabv3-semantic-scene-segmentation	'images307.zip?dl=0'   sample_data
 images307				 input


In [0]:
from google.colab import files
files.download('00_object_ratios.csv')